In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix, coo_matrix

# Детерминированные алгоритмы
from implicit.nearest_neighbours import ItemItemRecommender, CosineRecommender, TFIDFRecommender, BM25Recommender

# Метрики
from implicit.evaluation import train_test_split
from implicit.evaluation import precision_at_k, mean_average_precision_at_k, AUC_at_k, ndcg_at_k

In [2]:
data = pd.read_csv('./webinar_2/retail_train.csv')
data.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [3]:
test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

### Задание 1. Weighted Random Recommendation

Напишите код для случайных рекоммендаций, в которых вероятность рекомендовать товар прямо пропорциональна логарифму продаж
- Можно сэмплировать товары случайно, но пропорционально какому-либо весу
- Например, прямопропорционально популярности. Вес = log(sales_sum товара)

In [4]:
popular = data_train.groupby('item_id')['sales_value'].sum().reset_index()
popular.sort_values('sales_value', ascending=False, inplace=True)

popular['weight'] = popular.sales_value / popular.sales_value.sum()
popular.drop(columns='sales_value', inplace=True)

popular.head()

,item_id,weight
55470,6534178,0.063498
55430,6533889,0.005741
28895,1029743,0.005071
55465,6534166,0.004278
34707,1082185,0.003691


In [5]:
def weighted_random_recommendation(items_weights, n=5):
    """Случайные рекоммендации
    
    Input
    -----
    items_weights: pd.DataFrame
        Датафрейм со столбцами item_id, weight. Сумма weight по всем товарам = 1
    """
    
    items = items_weights.item_id.values
    probs = items_weights.weight.values
    
    recs = np.random.choice(items, size=n, p=probs, replace=False)
    
    return recs.tolist()

In [7]:
%%time

result2 = data_test.groupby('user_id')['item_id'].unique().reset_index()
result2.columns=['user_id', 'actual']
result2.head(2)

items = data_train.item_id.unique()

result2['weighted_random_recommendation'] = result2['user_id'].apply(lambda x: weighted_random_recommendation(popular, n=5))
result2.head(2)

CPU times: user 3.15 s, sys: 208 ms, total: 3.36 s
Wall time: 3.45 s


,user_id,actual,weighted_random_recommendation
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[820341, 1131400, 1106210, 6773204, 924770]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[929410, 843222, 952593, 5569261, 995916]"


### Задание 2. Расчет метрик
Рассчитайте Precision@5 для каждого алгоритма с помощью функции из вебинара 1. Какой алгоритм показывает лучшее качество?

In [36]:
result = pd.read_csv('./webinar_2/predictions_basic.csv', converters={'random_recommendation': eval,
                                                                      'popular_recommendation': eval,
                                                                      'itemitem': eval,
                                                                      'cosine': eval,
                                                                      'tfidf': eval,
                                                                      'own_purchases': eval})

result['weighted_random_recommendation'] = result['user_id'].apply(lambda x: weighted_random_recommendation(popular, n=5))
result['actual'] = result2['actual']

result.head(2)

,user_id,actual,random_recommendation,popular_recommendation,weighted_random_recommendation,itemitem,cosine,tfidf,own_purchases
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[1265481, 5995177, 831470, 1726294, 38552]","[6534178, 6533889, 1029743, 6534166, 1082185]","[1106872, 10283062, 13213524, 948793, 1117039]","[981760, 1127831, 1098066, 826249, 878996]","[981760, 1127831, 1098066, 878996, 826249]","[981760, 1127831, 1098066, 826249, 878996]","[999999, 1082185, 1029743, 995785, 1004906]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[2266886, 15563625, 6978868, 5653979, 833545]","[6534178, 6533889, 1029743, 6534166, 1082185]","[8249550, 1051394, 13095563, 1134414, 1062467]","[981760, 995242, 1029743, 840361, 961554]","[981760, 1004906, 961554, 1096036, 1080414]","[981760, 1004906, 859075, 1096036, 961554]","[999999, 1082185, 1098066, 6534178, 1127831]"


In [37]:
def precision_at_k_(recommended_list, bought_list, k=5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    bought_list = bought_list
    recommended_list = recommended_list[:k]
    
    flags = np.isin(bought_list, recommended_list)
    
    precision = flags.sum() / len(recommended_list)
    
    
    return precision

In [50]:
result.columns[2:]

Index(['random_recommendation', 'popular_recommendation',
       'weighted_random_recommendation', 'itemitem', 'cosine', 'tfidf',
       'own_purchases'],
      dtype='object')

In [55]:
precisions = []
models = []
for column in result.columns[2:]:
    precision = result.apply(lambda x: precision_at_k_(x[column], x['actual']), axis=1).mean()
    precisions.append(precision)
    models.append(column)

metrics = pd.DataFrame(data={'models': models, 'presision@5': precisions}, columns=['models', 'presision@5'])
metrics

,models,presision@5
0,random_recommendation,0.000294
1,popular_recommendation,0.155240
2,weighted_random_recommendation,0.021645
3,itemitem,0.033595
4,cosine,0.035260
5,tfidf,0.036141
6,own_purchases,0.179693


Самый лучший результат показал алгоритм, рекомендующий собственный покупки пользователя, на втором месте по метрике алгоритм на основе популярных рекомендаций

### Задание 3*. Улучшение бейзлайнов и ItemItem

- Попробуйте улучшить бейзлайны, считая их на топ- {2000, 5000, 10000} товаров. Постройте график зависимости precision@5 от топ-*
- Попробуйте улучшить разные варианты ItemItemRecommender, выбирая число соседей $K$. Постройте график зависимости precision@5 от $K$ 

In [ ]:
# your_code